In [ ]:
from tqdm import tqdm, trange
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import umap.umap_ as umap


import torch
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from transformers import AutoModel, AutoImageProcessor

import clip
from diffusers import StableDiffusionInstructPix2PixPipeline

from attacker_network import AttackerNetwork


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root='dogs_data/Images/', transform=transform)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

In [ ]:
dino = AutoModel.from_pretrained("facebook/dinov2-small")
dino.to(device)
dino.eval()
for p in dino.parameters():
    p.requires_grad = False

clip_model = clip.load("ViT-L/14", device=device)[0]
clip_model.eval()


pix2pix = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    safety_checker=None,
).to(device)

for component in [pix2pix.vae, pix2pix.unet, pix2pix.text_encoder]:
    for param in component.parameters():
        param.requires_grad = False
    component.eval()

pix2pix.unet.enable_gradient_checkpointing()

In [ ]:
def compute_centroids(model):
    features = []
    labels = []

    for images, label in tqdm(dataloader):
        images = images.to(device)
        with torch.no_grad():
            feature = model(images).last_hidden_state[:, 0].cpu().numpy()
        features.append(feature)
        labels.extend(label.cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.array(labels)

    unique_classes = np.unique(labels)
    n_classes = len(unique_classes)

    centroids = np.zeros((n_classes, features.shape[1]))
    for i, c in enumerate(unique_classes):
        class_indices = np.where(labels == c)[0]
        centroids[i] = np.mean(features[class_indices], axis=0)

    os.makedirs("variables", exist_ok=True)
    np.save('variables/centroids.npy', centroids)
    np.save('variables/features.npy', features)
    np.save('variables/labels.npy', labels)

    return centroids, features, labels

In [ ]:
# centroids, features, labels = compute_centroids(dino)

centroids = np.load('variables/centroids.npy')
features = np.load('variables/features.npy')
labels = np.load('variables/labels.npy')

In [ ]:
np.random.seed(0)

unique_classes = np.unique(labels)
n_classes = len(unique_classes)

class_id_to_name = {v: k for k, v in dataset.class_to_idx.items()}
class_names = [class_id_to_name[cls] for cls in unique_classes]

random_classes = np.random.choice(unique_classes, size=2, replace=False)
starting_class = random_classes[0]
starting_class_name = class_id_to_name[starting_class].split('-')[-1]
target_class = random_classes[1]
target_class_name = class_id_to_name[target_class].split('-')[-1]

classes = {starting_class: starting_class_name + " (starting)", target_class: target_class_name + " (target)"}

print(f"Starting class: {starting_class_name} ({starting_class})")
print(f"Target class: {target_class_name} ({target_class})")

In [ ]:
np.random.seed(0)

subset_size = 100

starting_features = []
target_features = []
for c in random_classes:
    class_indices = np.where(labels == c)[0]
    sampled_indices = np.random.choice(class_indices, size=subset_size, replace=False)
    if c == starting_class:
        starting_features.append(features[sampled_indices])
    else:
        target_features.append(features[sampled_indices])

starting_features = np.concatenate(starting_features, axis=0)
target_features = np.concatenate(target_features, axis=0)

starting_labels = np.full((starting_features.shape[0],), starting_class)
target_labels = np.full((target_features.shape[0],), target_class)

plotting_features = np.concatenate([starting_features, target_features], axis=0)
plotting_labels = np.concatenate([starting_labels, target_labels], axis=0)

umap_reducer = umap.UMAP(n_components=2, random_state=0)
embedding = umap_reducer.fit_transform(plotting_features)

colors = cm.rainbow(np.linspace(0, 1, len(random_classes)))

plt.figure(figsize=(10, 8))
# Plot features for each selected class with a distinct color
for i, cls in enumerate(random_classes):
    cls_mask = (plotting_labels == cls)
    plt.scatter(
        embedding[cls_mask, 0],
        embedding[cls_mask, 1],
        s=50,
        color=colors[i],
        label=classes[cls]
    )

plt.title("UMAP Projection of Image Features for Starting Class and Target Class")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.legend()
plt.show()

In [ ]:
starting_centroid = centroids[starting_class]
target_centroid = centroids[target_class]
starting_centroid = starting_centroid / np.linalg.norm(starting_centroid)
target_centroid = target_centroid / np.linalg.norm(target_centroid)

# Compute the distance between the centroids
distance = cosine_distances(starting_centroid.reshape(1, -1), target_centroid.reshape(1, -1))[0][0]
print(f"Distance between centroids: {distance:.2f}")

distances = cosine_distances(centroids)
average_distance = distances.mean()
print(f"Average distance between centroids: {average_distance:.2f}")

min_distance = distances[~np.eye(distances.shape[0], dtype=bool)].min()
print(f"Minimum distance between centroids: {min_distance:.2f}")

max_distance = distances[~np.eye(distances.shape[0], dtype=bool)].max()
print(f"Maximum distance between centroids: {max_distance:.2f}")

In [ ]:
# target_indices = [i for i, (_, label) in enumerate(dataset) if label == starting_class]
# target_indices = np.save('variables/target_indices.npy', target_indices)
target_indices = np.load('variables/target_indices.npy')
breed_subset = Subset(dataset, target_indices)

pil_to_tensor = transforms.ToTensor() 

In [ ]:
img = breed_subset[0][0]
img = img.unsqueeze(0).to(device)
feat = dino(img).last_hidden_state[:, 0].cpu().numpy()


In [ ]:
cosine_distances(starting_centroid.reshape(1, -1), feat.reshape(1, -1))[0][0]

In [ ]:
np.random.seed(0)

attacker = AttackerNetwork(
    device=device,
    pix2pix_model=pix2pix,
    dinov2_model=dino,
    clip_model=clip_model,
)

n_epochs = 1

optimizer = torch.optim.Adam(attacker.parameters(), lr=1e-3)

target_centroid = torch.tensor(target_centroid).to(device).unsqueeze(0)
target_centroid = F.normalize(target_centroid, dim=1)

In [ ]:
image = breed_subset[0][0]


In [ ]:
image = image.to(device)

output, _, _ = attacker(image)

In [ ]:
output = F.normalize(output, dim=1)

cosine_sim = F.cosine_similarity(output.squeeze(0), target_centroid, dim=1)
cosine_dist = 1 - cosine_sim

loss = cosine_dist.mean()


In [ ]:
loss

In [ ]:


optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Loss: {loss.item():.4f}")

In [ ]:
for epoch in range(n_epochs):
    for image, _ in tqdm(breed_subset, desc=f"Epoch {epoch+1}/{n_epochs}"):
        image = image.to(device)

        output, _, _ = attacker(image)
        output_norm = F.normalize(output, dim=1)


        cosine_sim = F.cosine_similarity(output_norm, target_centroid, dim=1)
        cosine_dist = 1 - cosine_sim

        loss = cosine_dist.mean()
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item():.4f}")
